In [1]:
with open('names.txt', 'r') as f:
    words = f.read().splitlines()

In [2]:
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [3]:
# Trigrams
t = {}

for w in words:
    for ch1, ch2, ch3 in zip(w, w[1:], w[2:]):
        trigram = (ch1, ch2, ch3)
        t[trigram] = t.get(trigram, 0) + 1
        

In [4]:
t

{('e', 'm', 'm'): 100,
 ('m', 'm', 'a'): 72,
 ('o', 'l', 'i'): 69,
 ('l', 'i', 'v'): 54,
 ('i', 'v', 'i'): 78,
 ('v', 'i', 'a'): 147,
 ('a', 'v', 'a'): 161,
 ('i', 's', 'a'): 142,
 ('s', 'a', 'b'): 76,
 ('a', 'b', 'e'): 173,
 ('b', 'e', 'l'): 201,
 ('e', 'l', 'l'): 822,
 ('l', 'l', 'a'): 337,
 ('s', 'o', 'p'): 21,
 ('o', 'p', 'h'): 37,
 ('p', 'h', 'i'): 61,
 ('h', 'i', 'a'): 81,
 ('c', 'h', 'a'): 236,
 ('h', 'a', 'r'): 329,
 ('a', 'r', 'l'): 287,
 ('r', 'l', 'o'): 44,
 ('l', 'o', 't'): 14,
 ('o', 't', 't'): 34,
 ('t', 't', 'e'): 121,
 ('m', 'i', 'a'): 95,
 ('a', 'm', 'e'): 226,
 ('m', 'e', 'l'): 188,
 ('e', 'l', 'i'): 537,
 ('l', 'i', 'a'): 518,
 ('a', 'r', 'p'): 8,
 ('r', 'p', 'e'): 5,
 ('p', 'e', 'r'): 77,
 ('e', 'v', 'e'): 142,
 ('v', 'e', 'l'): 76,
 ('e', 'l', 'y'): 353,
 ('l', 'y', 'n'): 976,
 ('a', 'b', 'i'): 76,
 ('b', 'i', 'g'): 15,
 ('i', 'g', 'a'): 35,
 ('g', 'a', 'i'): 18,
 ('a', 'i', 'l'): 259,
 ('e', 'm', 'i'): 160,
 ('m', 'i', 'l'): 259,
 ('i', 'l', 'y'): 203,
 ('l', 'i',

Extracting unique characters from dataset

In [7]:
string = ""
for w in words:
    string += w

chars = sorted(list(set(list(string))))

Mapping the characters to index and index to characters. <br>
'.' is a special character which denotes start or end of the word <br>
index for '.' is 0

In [11]:
itoc = {i+1:c for i,c in enumerate(chars)}
itoc[0] = '.'
ctoi = {c:i for i,c in itoc.items()}

Save the counts of these trigram is 27 * 27 * 27 matrix <br>
and then we will calculate the probability distribution for these trigrams

In [25]:
import torch

N = torch.zeros((27, 27, 27), dtype=torch.int32)

In [26]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        idx1, idx2, idx3 = ctoi[ch1], ctoi[ch2], ctoi[ch3]
        N[idx1, idx2, idx3] += 1


Adding 1 to all the values. Purpose of adding the 1 to values <br>
1. when calculating the log of value for calculation of log-likelihood, log(0) would be -inf <br>
    In order to avoid that, we'll add 1 to all the values
2. Moreover, it also adds smoothing affect to model

In [27]:
N = N + 1
N

tensor([[[  1,   1,   1,  ...,   1,   1,   1],
         [  1, 208, 191,  ...,  28, 174, 153],
         [  1, 170,   1,  ...,   1,   5,   1],
         ...,
         [  1,  58,   1,  ...,   2,  18,  12],
         [  1, 247,   1,  ...,   1,   1,   3],
         [  1, 457,   1,  ...,   1,  92,   2]],

        [[  1,   1,   1,  ...,   1,   1,   1],
         [ 41,   1,   6,  ...,   1,  21,  12],
         [ 37,  29,  21,  ...,   1,  13,   1],
         ...,
         [ 12,   6,   1,  ...,  18,   7,   4],
         [164, 390,  14,  ...,   1,  17,  41],
         [ 39, 124,   1,  ...,   1,  13,  23]],

        [[  1,   1,   1,  ...,   1,   1,   1],
         [ 47,   6,   6,  ...,   5,  32,   5],
         [  2,   9,   1,  ...,   1,  10,   1],
         ...,
         [  1,   1,   1,  ...,   1,   1,   1],
         [ 56,   5,   2,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1]],

        ...,

        [[  1,   1,   1,  ...,   1,   1,   1],
         [ 11,   1,   3,  ...,   1,  11,   1]

we will calculate the probabilities along the third dimesion <br>
N matrix is having 27 * 27 * 27 dimensions <br>
when you sum along the third dimension and divides the numbers in third dimesion with this sum, you calculate; <br>
- given the characters in 1st dimension and 2nd dimension, what is the probability distribution for 3 character <br>

we need to set keepdim parameter as True, as it set the dimension as 1 * 27 instead of just 27 <br>
so according to broadcasting rule; (27 * 27 * 27) times (1 * 27) would lead to matrix having (27 * 27 * 27) dimensions 



In [50]:
P = N.float()
P /= P.sum(2, keepdim=True)

Giving initial two characters to the model and it is predicting the third character

In [86]:
# g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = ['j', 'u']
    ix1 = 10
    ix2 = 21
    while True:
        p = P[ix1, ix2]
        ix1 = ix2

        # select the index of third character wrt probability distribution along that axis
        ix2 = torch.multinomial(p, num_samples=1, replacement=True).item()
        out.append(itoc[ix2])
        # print(itos[ix])
        if ix2 == 0:
            break
    print(''.join(out))

julian.
jus.
juleik.
juekalienylilena.
jus.
julanielayn.
julzmre.
jujulsynd.
julyah.
juna.


## Model Evaluation

- probability of any character in untrained model would be 1/27 which is 4%
so anything above 4% is good in evaluation
- GOAL: Likelihood is used in this statistical modeling and Likelihood is product of the probability and should be as high as possible
- But since this product is very tiny number, we need to work on log likelihood 
- log(a*b*c) = log(a) + log(b) + log(c)  <br>
so we will adding these log probability for calculation of log likelihood <br>
- Loss function is negative log likelihood and needs to be minimized

In [71]:
log_likelihood = 0
n = 0

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        idx1, idx2, idx3 = ctoi[ch1], ctoi[ch2], ctoi[ch3]
        prob = P[idx1, idx2, idx3]
        log_prob = torch.log(prob)
        log_likelihood += log_prob
        n += 1

print(f"Log Likelihood: {log_likelihood}")
print(f"Negative Log Likelihood: {-log_likelihood}")
print(f"Average NLL: {-log_likelihood/n:.4f}")

Log Likelihood: 956928.5625
Negative Log Likelihood: -956928.5625
Average NLL: -4.8795
